In [ ]:
from plyfile import PlyData

import numpy as np

import plotly.graph_objs as go
from plotly.subplots import make_subplots

from sklearn.cluster import DBSCAN
from sklearn.neighbors import KernelDensity

from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

import open3d as o3d


## Read Point Cloud

In [ ]:
def read_ply(file_path):
    plydata = PlyData.read(file_path)
    x = plydata['vertex']['x']
    y = plydata['vertex']['y']
    z = -plydata['vertex']['z']
    r = plydata['vertex']['red']
    g = plydata['vertex']['green']
    b = plydata['vertex']['blue']
    colors = np.array([r, g, b]).T
    points = np.array([x, y, z]).T
    print(f"Number of points: {points.shape[0]}")
    return points, colors

## Plot Original Point Cloud

In [ ]:
def plot_point_cloud(points, colors=None):
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
    fig.add_trace(go.Scatter3d(x=points[:, 0], y=points[:, 1], z=points[:, 2], mode='markers', marker=dict(size=1, color=colors)), row=1, col=1)
    fig.update_layout(width=1000, height=1000)
    fig.show()

## Density

In [ ]:
def dbscan(points, eps=0.5, min_samples=200):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(points)
    return labels

def get_peaks(density, plot=False):
    density_values, bin_edges = np.histogram(density, bins=100) 
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    smoothed_density = gaussian_filter1d(density_values, sigma=2)

    # Step 2: Detect peaks in the smoothed histogram
    peaks, _ = find_peaks(smoothed_density, height=20)  # Adjust height to filter out small peaks

    # Step 3: Identify start and end of each peak
    peak_boundaries = []
    for peak in peaks:
        start, end = peak, peak

        # Move left to find where density starts increasing towards the peak
        while start > 0 and smoothed_density[start - 1] < smoothed_density[start]:
            start -= 1
        
        # Move right to find where density starts decreasing after the peak
        while end < len(smoothed_density) - 1 and smoothed_density[end + 1] < smoothed_density[end]:
            end += 1

        peak_boundaries.append((start, end))

    if plot:
        fig = go.Figure()

        # Original histogram
        fig.add_trace(go.Scatter(
            x=bin_centers,
            y=density_values,
            mode='lines',
            name="Original Histogram"
        ))

        # Smoothed line
        fig.add_trace(go.Scatter(
            x=bin_centers,
            y=smoothed_density,
            mode='lines',
            line=dict(color='orange'),
            name="Smoothed Histogram"
        ))

        # Detected peaks
        fig.add_trace(go.Scatter(
            x=bin_centers[peaks],
            y=smoothed_density[peaks],
            mode='markers',
            marker=dict(color='red', size=10, symbol='x'),
            name="Detected Peaks"
        ))

        # Peak boundaries
        for idx, (start, end) in enumerate(peak_boundaries):
            fig.add_trace(go.Scatter(
                x=[bin_centers[start], bin_centers[start]],
                y=[0, max(density_values)],
                mode='lines',
                line=dict(color='green', dash='dash'),
                showlegend=idx == 0,
                name="Peak Start" if idx == 0 else None
            ))
            fig.add_trace(go.Scatter(
                x=[bin_centers[end], bin_centers[end]],
                y=[0, max(density_values)],
                mode='lines',
                line=dict(color='purple', dash='dash'),
                showlegend=idx == 0,
                name="Peak End" if idx == 0 else None
            ))

        # Layout
        fig.update_layout(
            title="Density Histogram with Peak Detection",
            xaxis_title="Density",
            yaxis_title="Number of Points",
            legend=dict(
                x=1.0,                # Position legend at the far right
                y=1.0,                # Position legend at the top
                xanchor='right',      # Anchor the right side of the legend box to the x position
                yanchor='top',        # Anchor the top of the legend box to the y position
                bgcolor='rgba(255, 255, 255, 0.5)',  # Optional: Set a semi-transparent background for better visibility
                bordercolor='Black', # Optional: Add a border color to the legend
                borderwidth=1         # Optional: Set the border width
            )
        )

        fig.show()
    return peak_boundaries, bin_centers


def monte_carlo_kde(points: np.ndarray, bandwidth: float, sample_size: int = 500) -> np.ndarray:
    sample_indices = np.random.choice(len(points), sample_size, replace=False)
    sample_points = points[sample_indices]
    
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(sample_points)
    
    log_density = kde.score_samples(points)
    density = np.exp(log_density)
    
    return density

def get_densest_cluster(points, colors=None, plot=False):
    density = monte_carlo_kde(points, bandwidth=1, sample_size=1000)  
    peak_boundaries, bin_centers = get_peaks(density, plot=True)
    first_peak_end_index = peak_boundaries[0][1]
    first_peak_end = bin_centers[first_peak_end_index]
    fig = go.Figure()
    points = points[density > first_peak_end]
    density = density[density > first_peak_end]

    if plot:
        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(
                size=5,
                # color uses r, g, and b
                color=density,
                colorscale='Viridis',
                colorbar=dict(title='Density'),
                opacity=0.8
            )
        ))

        fig.update_layout(
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            title="3D Point Cloud with Density Color-Coding"
        )

        fig.show()
    return points#, colors


## Floor Separation

In [ ]:

from scipy.spatial.transform import Rotation

def find_floor_plane(points, distance_threshold=0.02, min_floor_points=100):
    """Find the floor plane in a point cloud."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    plane_model, inliers = pcd.segment_plane(
        distance_threshold=distance_threshold,
        ransac_n=3,
        num_iterations=1000
    )
    

    inlier_mask = np.zeros(len(points), dtype=bool)
    inlier_mask[inliers] = True
    
    print(f"Number of inlier indices: {len(inliers)}")
    print(f"Number of True values in inliers: {np.sum(inliers)}")
    
    floor_points = points[inlier_mask]
    non_floor_points = points[~inlier_mask]
    
    print(f"Number of floor points: {len(floor_points)}")
    print(f"Number of non-floor points: {len(non_floor_points)}")
    
    if len(floor_points) < min_floor_points:
        print(f"Warning: Found only {len(floor_points)} floor points. Might be unreliable.")
    
    return floor_points, non_floor_points, plane_model


def find_optimal_threshold(points, 
                           initial_threshold=0.01, 
                           max_threshold=0.3, 
                           iterations=50):
    """
    Automatically find optimal distance threshold for floor detection without predefined floor ratio bounds.
    The function iteratively adjusts the threshold and monitors the change in floor_ratio to determine when to stop.
    
    Parameters:
    -----------
    points : np.ndarray
        Input point cloud as a NumPy array of shape (N, 3).
    initial_threshold : float, default=0.02
        Starting distance threshold value.
    max_threshold : float, default=0.1
        Maximum allowed threshold.
    iterations : int, default=10
        Maximum number of iterations for searching the optimal threshold.
    
    Returns:
    --------
    best_threshold : float
        The optimal distance threshold found.
    best_ratio : float
        The ratio of floor points corresponding to the optimal threshold.
    stats : dict
        Dictionary containing statistics about the threshold search process.
    """
    total_points = len(points)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # Calculate point cloud statistics
    distances = np.asarray(pcd.compute_nearest_neighbor_distance())
    median_distance = np.median(distances)
    
    # Initialize threshold search
    threshold = initial_threshold
    best_threshold = threshold
    best_ratio = 0
    
    stats = {
        'iterations': [],
        'thresholds': [],
        'floor_ratios': [],
        'improvements': [],
        'median_distance': median_distance
    }
    
    for iteration in range(iterations):
        # Segment plane with current threshold
        plane_model, inliers = pcd.segment_plane(
            distance_threshold=threshold,
            ransac_n=3,
            num_iterations=1000
        )
        
        floor_ratio = len(inliers) / total_points
        
        # Store statistics
        stats['iterations'].append(iteration)
        stats['thresholds'].append(threshold)
        stats['floor_ratios'].append(floor_ratio)        

        if threshold >= max_threshold:
            print(f"Stopping search: Reached maximum threshold {max_threshold}")
            break

        threshold += (max_threshold - initial_threshold) / iterations
        threshold = min(threshold, max_threshold)
    
    smoothed_ratios = gaussian_filter1d(stats['floor_ratios'], sigma=3)
    second_derivative = np.gradient(np.gradient(smoothed_ratios))
    # best threshold is the one with the lowest second derivative
    best_threshold = stats['thresholds'][np.argmin(second_derivative)]
    best_ratio = stats['floor_ratios'][np.argmin(second_derivative)]
    

    # Collect final statistics
    stats['optimal_threshold'] = best_threshold
    stats['final_floor_ratio'] = best_ratio
    stats['median_point_distance'] = median_distance
    
    return best_threshold, best_ratio, stats

def find_floor_plane_auto(points, min_floor_points=100, visualize_threshold_search=False):
    """
    Enhanced floor detection with automatic threshold selection.
    """
    optimal_threshold, floor_ratio, stats = find_optimal_threshold(points)
    
    if visualize_threshold_search:
        # Create visualization of threshold search
        fig = go.Figure()
        
        # Plot threshold evolution
        fig.add_trace(go.Scatter(
            x=stats['iterations'],
            y=stats['thresholds'],
            name='Threshold',
            mode='lines+markers',
            line=dict(color='red'),
        ))
        
        # Plot floor ratio evolution
        fig.add_trace(go.Scatter(
            x=stats['iterations'],
            y=stats['floor_ratios'],
            name='Floor Ratio',
            mode='lines+markers',
            line=dict(color='cyan'),
            yaxis='y2'
        ))

        smoothed_floor_ratio = gaussian_filter1d(stats['floor_ratios'], sigma=3)
        fig.add_trace(go.Scatter(
            x=stats['iterations'],
            y=smoothed_floor_ratio,
            name='Smoothed Floor Ratio',
            mode='lines',
            line=dict(color='blue'),
            yaxis='y2'
        ))


        # plot 2nd derivative of floor ratio
        fig.add_trace(go.Scatter(
            x=stats['iterations'],
            y=np.gradient(np.gradient(smoothed_floor_ratio)),
            name='2nd Derivative of Floor Ratio',
            mode='lines+markers',
            line=dict(color='blue'),
        ))
        
        # Plot floor ratio evolution
        fig.add_trace(go.Scatter(
            x=stats['iterations'],
            y=stats['improvements'],
            name='Improvements',
            mode='lines+markers',
            line=dict(color='blue'),
            yaxis='y2'
        ))
        
        fig.update_layout(
            title='Threshold Search Evolution',
            xaxis_title='Iteration',
            yaxis_title='Threshold',
            yaxis2=dict(
                title='Floor Ratio',
                overlaying='y',
                side='right'
            )
        )
        fig.show()
    
    print(f"Found optimal threshold: {optimal_threshold:.4f}")
    print(f"Floor ratio: {floor_ratio:.2%}")
    print(f"Median point distance: {stats['median_point_distance']:.4f}")
    
    # Use the optimal threshold to find the floor
    return find_floor_plane(points, distance_threshold=optimal_threshold, 
                          min_floor_points=min_floor_points)


def determine_model_orientation(points, plane_model):
    """Determine if the model is upside down relative to the floor plane."""
    a, b, c, d = plane_model
    normal_vector = np.array([a, b, c])
    
    # Calculate signed distances to the plane
    signed_distances = (points @ normal_vector + d)
    
    points_above = np.sum(signed_distances > 0)
    points_below = np.sum(signed_distances < 0)
    total_points = len(points)
    
    is_upside_down = (points_below / total_points) > 0.2
    
    orientation_info = {
        "total_points": total_points,
        "points_above_floor": points_above,
        "points_below_floor": points_below,
        "ratio_above": points_above / total_points,
        "ratio_below": points_below / total_points,
        "is_upside_down": is_upside_down,
        "floor_normal": normal_vector
    }
    
    return orientation_info

def align_to_xy_plane(points, plane_model, orientation_info):
    """Align the point cloud so the floor is parallel to the XY plane and positioned at z=0."""
    # Extract plane parameters
    a, b, c, d = plane_model
    floor_normal = np.array([a, b, c])
    
    # If the model is upside down, flip the normal
    if orientation_info["is_upside_down"]:
        floor_normal = -floor_normal
    
    # Define the target normal (Z-axis)
    z_axis = np.array([0, 0, 1])
    
    # Calculate rotation required to align floor_normal with Z-axis
    rotation_axis = np.cross(floor_normal, z_axis)
    norm_rotation_axis = np.linalg.norm(rotation_axis)
    
    if norm_rotation_axis < 1e-6:
        # The normals are already aligned or opposite
        if np.dot(floor_normal, z_axis) < 0:
            rotation_matrix = -np.eye(3)
        else:
            rotation_matrix = np.eye(3)
    else:
        rotation_axis /= norm_rotation_axis
        rotation_angle = np.arccos(np.clip(np.dot(floor_normal, z_axis), -1.0, 1.0))
        rotation = Rotation.from_rotvec(rotation_angle * rotation_axis)
        rotation_matrix = rotation.as_matrix()
    
    # Rotate all points
    rotated_points = (rotation_matrix @ points.T).T
    
    # Find a point on the original plane
    plane_norm_sq = a**2 + b**2 + c**2
    if plane_norm_sq == 0:
        raise ValueError("Invalid plane model with zero normal vector.")
    p0 = np.array([-a * d / plane_norm_sq,
                   -b * d / plane_norm_sq,
                   -c * d / plane_norm_sq])
    
    # Rotate the point on the plane
    p0_rotated = rotation_matrix @ p0
    
    # Calculate translation to bring the rotated plane to z=0
    translation_z = -p0_rotated[2]
    translation = np.array([0, 0, translation_z])
    
    # Apply translation
    aligned_points = rotated_points + translation
    
    return aligned_points, rotation_matrix, translation

def remove_points_below_floor(points):
    """Remove points below the floor (Z < threshold)."""
    above_floor_mask = points[:, 2] >= 0
    return points[above_floor_mask]

def denoise_point_cloud(points, neighbors=20, std_ratio=0.1):
    """Denoise the point cloud using statistical outlier removal."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=neighbors, std_ratio=std_ratio)
    return np.asarray(pcd.points)[ind]

def process_point_cloud(points, min_floor_points=500, distance_threshold=None):
    """Complete pipeline to process the point cloud."""
    result = {}
    # 1. Find floor
    
    if distance_threshold is None:
        floor_points, non_floor_points, plane_model = find_floor_plane_auto(
            points, 
            min_floor_points=min_floor_points,
            visualize_threshold_search=True
        )
    else:
        floor_points, non_floor_points, plane_model = find_floor_plane(points, distance_threshold=distance_threshold, 
                          min_floor_points=min_floor_points)
    result['floor_points'] = floor_points
    result['non_floor_points'] = non_floor_points
    result['plane_model'] = plane_model
    
    # 2. Determine model orientation
    orientation_info = determine_model_orientation(
        non_floor_points, 
        plane_model
    )
    
    # 3. Align to XY plane
    aligned_points, rotation_matrix, translation = align_to_xy_plane(
        non_floor_points, 
        plane_model, 
        orientation_info
    )
    result['aligned_points'] = aligned_points
    result['orientation_info'] = orientation_info
    result['transformation'] = {
        'rotation': rotation_matrix,
        'translation': translation
    }

    
    # 4. Remove points below floor
    final_points = remove_points_below_floor(aligned_points)
    result['final_points'] = final_points

    # 5. Denoise point cloud
    denoised_points = denoise_point_cloud(final_points)
    result['denoised_points'] = denoised_points

    # 6. Denoise point cloud DBSCAN
    dbscan_labels = dbscan(final_points, eps=0.9)
    result['dbscan_labels'] = dbscan_labels
    
    return result


## Visualize Floor Separation

In [ ]:
def plot_points_3d(points, color='blue', size=2, opacity=0.6):
    """Create a basic 3D scatter plot for points."""
    return go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(
            size=size,
            color=color,
            opacity=opacity
        ),
        name=f'Points ({len(points)} pts)'
    )

def plot_plane(plane_model, points, grid_size=20):
    """Create a surface plot for a plane within the points bounds."""
    a, b, c, d = plane_model
    
    # Get bounds from points
    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()
    
    # Create grid
    x = np.linspace(x_min, x_max, grid_size)
    y = np.linspace(y_min, y_max, grid_size)
    X, Y = np.meshgrid(x, y)
    
    # Calculate Z values for the plane
    Z = (-a * X - b * Y - d) / c
    
    return go.Surface(
        x=X, y=Y, z=Z,
        opacity=0.3,
        showscale=False,
        name='Floor plane'
    )

def visualize_floor_detection(points, floor_points, non_floor_points, plane_model):
    """Visualize the floor detection step."""
    fig = go.Figure()
    
    # Add original points with low opacity
    fig.add_trace(plot_points_3d(points, color='gray', opacity=0.2))
    
    # Add floor points
    fig.add_trace(plot_points_3d(floor_points, color='green', opacity=0.8))
    
    # Add non-floor points
    fig.add_trace(plot_points_3d(non_floor_points, color='red', opacity=0.8))
    
    # Add floor plane
    fig.add_trace(plot_plane(plane_model, points))
    
    fig.update_layout(
        title='Floor Detection Results',
        scene=dict(
            aspectmode='data'
        ),
        showlegend=True
    )
    
    return fig

def visualize_orientation(points, plane_model, orientation_info):
    """Visualize the model orientation relative to the floor."""
    # Split points based on their position relative to the floor
    a, b, c, d = plane_model
    signed_distances = (points @ np.array([a, b, c]) + d)
    
    points_above = points[signed_distances > 0]
    points_below = points[signed_distances < 0]
    
    fig = go.Figure()
    
    # Add points above floor
    if len(points_above) > 0:
        fig.add_trace(plot_points_3d(points_above, color='blue', opacity=0.8))
    
    # Add points below floor
    if len(points_below) > 0:
        fig.add_trace(plot_points_3d(points_below, color='red', opacity=0.8))
    
    # Add floor plane
    fig.add_trace(plot_plane(plane_model, points))
    
    # Add floor normal vector at center of points
    center = points.mean(axis=0)
    normal = orientation_info['floor_normal'] * (points.max() - points.min()).mean() * 0.2
    
    fig.add_trace(go.Scatter3d(
        x=[center[0], center[0] + normal[0]],
        y=[center[1], center[1] + normal[1]],
        z=[center[2], center[2] + normal[2]],
        mode='lines+markers',
        line=dict(color='black', width=5),
        name='Floor normal'
    ))
    
    fig.update_layout(
        title=f"Model Orientation (Upside down: {orientation_info['is_upside_down']})",
        scene=dict(aspectmode='data'),
        showlegend=True
    )
    
    return fig

def visualize_alignment(original_points, aligned_points):
    """Visualize the alignment transformation."""
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=('Original Points', 'Aligned Points')
    )
    
    # Original points
    fig.add_trace(
        plot_points_3d(original_points, color='blue'),
        row=1, col=1
    )
    
    # Aligned points
    fig.add_trace(
        plot_points_3d(aligned_points, color='green'),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title='Point Cloud Alignment Results',
        scene=dict(aspectmode='data'),
        scene2=dict(aspectmode='data'),
        showlegend=True
    )
    
    return fig

def visualize_final_result(original_points, final_points):
    """Visualize the original vs final processed point cloud."""
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=('Original Point Cloud', 'Processed Point Cloud')
    )
    
    # Original points
    fig.add_trace(
        plot_points_3d(original_points, color='blue'),
        row=1, col=1
    )
    
    # Final points
    fig.add_trace(
        plot_points_3d(final_points, color='green'),
        row=1, col=2
    )
    
    fig.update_layout(
        title='Final Processing Results',
        scene=dict(aspectmode='data'),
        scene2=dict(aspectmode='data'),
        showlegend=True
    )
    
    return fig

def visualize_complete_pipeline(result_dict):
    """Visualize all steps of the pipeline in a single figure."""
    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}],
               [{'type': 'scene'}, {'type': 'scene'}],
               [{'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=(
            'Floor Detection',
            'Orientation Analysis',
            'Alignment Result',
            'Final Result',
            'Denoised Result Statistical Outlier',
            'Denoised Result DBSCAN'
        )
    )
    
    # 1. Floor Detection
    floor_trace = plot_points_3d(result_dict['floor_points'], color='green')
    fig.add_trace(floor_trace, row=1, col=1)
    
    # 2. Orientation
    above_below_trace = plot_points_3d(result_dict['aligned_points'], color='blue')
    fig.add_trace(above_below_trace, row=1, col=2)
    
    # 3. Alignment
    aligned_trace = plot_points_3d(result_dict['aligned_points'], color='orange')
    fig.add_trace(aligned_trace, row=2, col=1)
    
    # 4. Final Result
    final_trace = plot_points_3d(result_dict['final_points'], color='red')
    fig.add_trace(final_trace, row=2, col=2)

    # 5. Denoised Result Statistical Outlier
    denoised_trace = plot_points_3d(result_dict['denoised_points'], color='purple')
    fig.add_trace(denoised_trace, row=3, col=1)

    # 6. Denoised Result DBSCAN
    # INSERT CODE HERE
    dbscan_labels = result_dict["dbscan_labels"]
    unique_labels = np.unique(dbscan_labels)
    
    for label in unique_labels:
        color = 'gray' if label == -1 else f"rgba({np.random.randint(0,255)},{np.random.randint(0,255)},{np.random.randint(0,255)},0.6)"
        label_points = result_dict['final_points'][dbscan_labels == label]
        fig.add_trace(
            plot_points_3d(label_points, color=color),
            row=3, col=2
        )
        
    fig.update_layout(
        title='Complete Pipeline Visualization',
        height=1000,
        showlegend=True
    )
    
    return fig

## Run Floor Separation

In [ ]:
def run_floor_separation(points, min_floor_points=500, distance_threshold=None):
    result = process_point_cloud(points, min_floor_points, distance_threshold)
    print(f"Floor points: {len(result['floor_points'])}, Non-floor points: {len(result['non_floor_points'])}")
    # Visualize individual steps
    fig_floor = visualize_floor_detection(
        points, 
        result['floor_points'], 
        result['non_floor_points'], 
        result['plane_model']
    )
    fig_floor.show()

    fig_orientation = visualize_orientation(
        points, 
        result['plane_model'], 
        result['orientation_info']
    )
    fig_orientation.show()

    fig_alignment = visualize_alignment(
        points, 
        result['aligned_points']
    )
    fig_alignment.show()

    fig_final = visualize_final_result(
        points, 
        result['final_points']
    )
    fig_final.show()

    # # Or visualize everything at once
    fig_complete = visualize_complete_pipeline(result)
    fig_complete.show()

### Bicycle

In [ ]:
bicycle_points, bicycle_colors = read_ply('data/points_bicycle.ply')
# plot_point_cloud(bicycle_points, bicycle_colors)
bicycle = get_densest_cluster(bicycle_points, plot=True)
run_floor_separation(bicycle)

### Bonsai

In [ ]:
bonsai_points, bonsai_colors = read_ply('data/points_bonsai.ply')
# plot_point_cloud(bonsai_points, bonsai_colors)
bonsai = get_densest_cluster(bonsai_points, plot=True)
run_floor_separation(bonsai)

### Counter

In [ ]:
counter_points, counter_colors = read_ply('data/points_counter.ply')
# plot_point_cloud(counter_points, counter_colors)
counter = get_densest_cluster(counter_points, plot=True)
run_floor_separation(counter)

### Garden

In [ ]:
garden_points, garden_colors = read_ply('data/points_garden.ply')
# plot_point_cloud(garden_points, garden_colors)
garden = get_densest_cluster(garden_points, plot=True)
run_floor_separation(garden)

### Kitchen

In [ ]:
kitchen_points, kitchen_colors = read_ply('data/points_kitchen.ply')
# plot_point_cloud(kitchen_points, kitchen_colors)
kitchen = get_densest_cluster(kitchen_points, plot=True)
run_floor_separation(kitchen)

### Room

In [ ]:
room_points, room_colors = read_ply('data/points_room.ply')
# plot_point_cloud(room_points, room_colors)
room = get_densest_cluster(room_points, plot=True)
run_floor_separation(room)

### Stump

In [ ]:
stump_points, stump_colors = read_ply('data/points_stump.ply')
# plot_point_cloud(counter_points, stump_colors)
stump = get_densest_cluster(stump_points, plot=True)
run_floor_separation(stump)